# Preparing volumetric data for the atlas website

To upload volumetric based data to the website you will need to provide
- A `volume` array containing the volume to be displayed. The volume must be registered to the Allen CCF mouse atlas.
- The names of the volume feature.


## Preparing your volume

Here we detail the format of the volume that must be prepared in order to upload it to the website

### 1. Register your volume to the Allen CCF

The website currently only supports mouse volumes that have been registered and aligned to the Allen CCF mouse brain. A number of useful tools exist for this process

These include,
- [brainreg](https://brainglobe.info/index.html)
- [brainregister](https://github.com/int-brain-lab/brainregister)
- [HERBS](https://github.com/Whitlock-Group/HERBS/tree/main)


### 2. Resample your volume to 25 um

The volume uploaded must have a resolution of 25 um in the ap, dv and ml dimensions. You can use the `simpleITK` python package to resample your data. This can be installed using `pip install SimpleITK`

The example below shows how you would resample a 10 um resolution volume to 25 um resolution.

In [ ]:
import SimpleITK as sitk
from pathlib import Path

# Path to original volume
img_path = Path.home().joinpath('path_to_volume', 'volume.tif')
save_img_path = img_path.parent.joinpath('volume_resampled.tif')

# Read in tiff image using simpleITK
reader = sitk.ImageFileReader()
reader.SetImageIO("TIFFImageIO")
reader.SetFileName(str(img_path))
img = reader.Execute()

# Resample the image
img_size = img.GetSize()
out_spacing = [2.5, 2.5, 2.5]
out_size = [int(img_size[0] / out_spacing[0]), int(img_size[1] / out_spacing[1]), int(img_size[2] / out_spacing[2])]
resample = sitk.ResampleImageFilter()
resample.SetOutputSpacing(out_spacing)
resample.SetSize(out_size)
resample.SetOutputDirection(img.GetDirection())
resample.SetOutputOrigin(img.GetOrigin())
resample.SetTransform(sitk.Transform())
resample.SetDefaultPixelValue(img.GetPixelIDValue())
resample.SetInterpolator(sitk.sitkBSpline)
new_img = resample.Execute(img)

# Save the image
# Write file to disk
writer = sitk.ImageFileWriter()
writer.SetImageIO("TIFFImageIO")
writer.SetFileName(str(save_img_path))
writer.Execute(new_img)

### 3. Adjust the axis of your volume

The orientation of the axis of the volume must be in the order ap, dv, ml. After resampling and ensuring the correct axis orientation the shape of your volume should be (528, 320, 456)

## Uploading an example volume

Here we show an example of uploading a compatible volume to both a local and remote bucket. To understand the difference between the two, please refer to the [getting started tutorial](https://github.com/int-brain-lab/iblbrainviewer/blob/main/docs/atlas_website_getting_started.ipynb#3.-Understand-some-useful-concepts).

The volume that we will upload is the Allen CCF 25um resolution diffusion weighted image (DWI) volume.

In [1]:
# Import ibl atlas module and load in the 25 um Allen CCF atlas
from iblatlas.atlas import AllenAtlas
brain_atlas = AllenAtlas(res_um=25)


# The DWI image is stored as an attribute called image
dwi = brain_atlas.image
print(dwi.shape)

(528, 456, 320)


If we explore the shape of our dwi image, we see that our volume has the correct size but the dimensions we need to switch the last two axis to match our expected volume shape of (528, 320, 456).

In [2]:
import numpy as np
dwi = np.swapaxes(dwi, 1, 2)
print(dwi.shape)

(528, 320, 456)


Now our volume is oriented correctly we can upload it to the website.

In [3]:
from iblbrainviewer.api import FeatureUploader
from pathlib import Path

# Instantiate local bucket
local_bucket_path = Path.home().joinpath('local_bucket')
up_local = FeatureUploader()

# Instantiate remote bucket
bucket = 'ibl_volume_example'
up_remote = FeatureUploader(bucket)


fname = 'dwi_allen_25'

# Upload locally
# For acronyms must provide hemisphere argument
up_local.local_volume(fname, dwi, output_dir=local_bucket_path)

# Upload remotely
if not up_remote.features_exist(fname):
    # For acronyms must provide hemisphere argument
    up_remote.upload_volume(fname, dwi)

url = up_remote.get_buckets_url([bucket])
print(url)

https://atlas.internationalbrainlab.org/?buckets=ibl_volume_example&bucket=ibl_volume_example
